In [ ]:
!pip install sentence-transformers
!pip install qdrant-client

In [ ]:
# pip install qdrant-client
# pip install sentence-transformers

import numpy as np
import pandas as pd
import re
import string
from sentence_transformers import SentenceTransformer

from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

In [ ]:
#QdrantSetup
client = QdrantClient(
    url="URL_DE_SEU_CLUSTER",
    api_key="SUA_API_KEY",
)

In [ ]:
# Carrega o dataset
taxonomy_ds = pd.read_hdf("subclass_2025_with_description.h5")
taxonomy_ds

,SECAO,SECAO_DESC,CLASS,CLASS_DESC,SUBCLASS,SUBCLASS_DESC,description_gpt_4o,description_gpt_4o_mini
0,A,HUMAN NECESSITIES,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,This subclass encompasses agricultural machine...,The subclass encompasses agricultural implemen...
1,A,HUMAN NECESSITIES,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A01C,PLANTING; SOWING; FERTILISING,"This subclass pertains to the methods, machine...",The subclass encompasses agricultural implemen...
2,A,HUMAN NECESSITIES,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A01D,HARVESTING; MOWING,This subclass pertains to agricultural machine...,This subclass encompasses agricultural impleme...
3,A,HUMAN NECESSITIES,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A01F,"THRESHING ; BALING OF STRAW, HAY OR THE LIKE; ...",The subclass pertains to the mechanical treatm...,This subclass encompasses inventions related t...
4,A,HUMAN NECESSITIES,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A01G,"HORTICULTURE; CULTIVATION OF VEGETABLES, FLOWE...",This subclass pertains to the specialized tech...,The subclass encompasses inventions related to...
...,...,...,...,...,...,...,...,...
648,H,ELECTRICITY,H10,SEMICONDUCTOR DEVICES; ELECTRIC SOLID-STATE DE...,H10F,INORGANIC SEMICONDUCTOR DEVICES SENSITIVE TO I...,This subclass pertains to the technical field ...,This subclass encompasses technologies related...
649,H,ELECTRICITY,H10,SEMICONDUCTOR DEVICES; ELECTRIC SOLID-STATE DE...,H10H,INORGANIC LIGHT-EMITTING SEMICONDUCTOR DEVICES...,This subclass pertains to the technical field ...,The subclass encompasses inventions related to...
650,H,ELECTRICITY,H10,SEMICONDUCTOR DEVICES; ELECTRIC SOLID-STATE DE...,H10K,ORGANIC ELECTRIC SOLID-STATE DEVICES,This subclass pertains to the field of nuclear...,The subclass encompasses technologies related ...
651,H,ELECTRICITY,H10,SEMICONDUCTOR DEVICES; ELECTRIC SOLID-STATE DE...,H10N,ELECTRIC SOLID-STATE DEVICES NOT OTHERWISE PRO...,This subclass pertains to microstructural tech...,The subclass encompasses inventions related to...


In [ ]:
limit=200
index_name = "patent-200" # Name of your Cluster

dimensions_768 = 768 #Size of embeddings
model_768 = SentenceTransformer('all-distilroberta-v1') #Loads the pre-trained SBERT model

dimensions_384 = 384 #Size of embeddings
model_384 = SentenceTransformer('all-MiniLM-L6-v2') #Loads the pre-trained SBERT model

dimensions = dimensions_384
model = model_384

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
client.recreate_collection(
    collection_name=index_name,
    vectors_config=VectorParams(size=dimensions, distance=Distance.COSINE),
)

<ipython-input-37-a1980b266d23>:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [ ]:
# Carrega o dataset
patent_ds = pd.read_json('2014_USPTO.json')
patent_ds

,Subclass_labels,Abstract,Title,No
0,"[A61B, G09B]",a method is presented to address quantitative ...,method and system for quantitative assessment ...,US08777630
1,[E21B],in a method for drilling a borehole real time ...,method for drilling a borehole,US08857538
2,"[C23C, H01J]",a magnetron actuator for moving a magnetron in...,homing device for magnetron rotating on two arms,US08900427
3,"[H05K, C23C, G06F]",a method of making a touch sensor assembly com...,touch sensor assembly and method of making the...,US08822833
4,"[C07D, A61K, A61P]",provided herein are novel epoxide inhibitors o...,epoxide inhibitors of cysteine proteases,US08673904
...,...,...,...,...
303329,[G06Q],methods systems and instructions stored on com...,universal payment module and system,US08762271
303330,[G06F],a semiconductor device includes a frequency se...,semiconductor device and control method for se...,US08656197
303331,[G08B],a warning system in which an audio pulse train...,warning system,US08760289
303332,[A61M],a rotational atherectomy device for abrading a...,rotational atherectomy device with fluid infla...,US08663195


In [ ]:
def filter_patents_by_subclass(df, search_string, limit=10):
  """Filters a DataFrame of patent data based on a search string.

  Args:
    df: A Pandas DataFrame containing patent data.
    search_string: The search string to match against subclass labels.
    limit: The maximum number of patents to return.

  Returns:
    A list of matching patents.
  """

  # Filter the DataFrame based on the search string
  #filtered_df = df[df['Subclass_labels'].apply(lambda labels:
  #                        any(search_string in label for label in labels))]

  filtered_df = df[df['Subclass_labels'].apply(lambda labels:
                          all(search_string in label for label in labels))]

  # Limit the results if necessary
  if limit:
    filtered_df = filtered_df.head(limit)

  # Convert the filtered DataFrame to a list of dictionaries
  matching_patents = filtered_df.to_dict('records')

  return matching_patents

In [ ]:
def clear_text(text):
    # Define a regex pattern for punctuation characters
    punctuation_pattern = f"[{re.escape(string.punctuation)}]"

    # Remove punctuation characters
    text_no_punctuation = re.sub(punctuation_pattern, '', text)

    # Replace multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', text_no_punctuation).strip()

    return cleaned_text.lower().strip()

In [ ]:
'''
text1 = 'a rock removal skid steer attachment includes a connection portion that has a rear face a lower edge and an upper edge the connection portion is configured for engaging a skid steer a pair of side plates extends from the connection portion the side plates are spaced from each other the distal section of the side plates relative to the connection portion includes a barbed portion each of the side plates has a top edge and a lower edge the top edges taper toward an associated one of the lower edges as the side plates extend away from the connection portion and towards the barbed portions the side plates are configured to be extended into a ground surface to extract boulders from the ground surface with the barbs'

text2 = 'The subclass encompasses agricultural implements and machines specifically designed for soil cultivation and preparation, including plowing, harrowing, and tilling. It covers various tools and devices utilized for breaking up, turning over, and aerating the soil to enhance agricultural productivity. This includes both manual and powered equipment, such as plows, cultivators, and rotary tillers, as well as attachments and accessories that improve their functionality. The subclass also addresses innovations in the design and operation of these implements, focusing on efficiency, effectiveness, and adaptability to different soil types and agricultural practices. Additionally, it considers the integration of technology in these tools, such as automation and precision farming techniques, aimed at optimizing soil management and crop yield. The classification reflects a comprehensive approach to the mechanization of agriculture, emphasizing advancements that contribute to sustainable farming practices.'

text3 = clear_text(text2)
print(text3)
#--------------------------
embedding = model_384.encode(text1).tolist()
print(embedding)

embedding = model_384.encode(text2).tolist()
print(embedding)

embedding = model_384.encode(text1+" "+text2).tolist()
print(embedding)

#-----------------------------

embedding = model_768.encode(text1).tolist()
print(embedding)

embedding = model_768.encode(text2).tolist()
print(embedding)

embedding = model_768.encode(text1+" "+text2).tolist()
print(embedding)

embedding = model_768.encode(text1+" "+text3).tolist()
print(embedding)
'''

'\ntext1 = \'a rock removal skid steer attachment includes a connection portion that has a rear face a lower edge and an upper edge the connection portion is configured for engaging a skid steer a pair of side plates extends from the connection portion the side plates are spaced from each other the distal section of the side plates relative to the connection portion includes a barbed portion each of the side plates has a top edge and a lower edge the top edges taper toward an associated one of the lower edges as the side plates extend away from the connection portion and towards the barbed portions the side plates are configured to be extended into a ground surface to extract boulders from the ground surface with the barbs\'\n\ntext2 = \'The subclass encompasses agricultural implements and machines specifically designed for soil cultivation and preparation, including plowing, harrowing, and tilling. It covers various tools and devices utilized for breaking up, turning over, and aeratin

In [ ]:
points = []
id=0;
patent_no=0;
title=''
subclass_labels=''
abstract=''
text=''

for index, row in taxonomy_ds.iterrows():

    subclass_id = row.SUBCLASS
    #subclass_genai_desc = clear_text(row.description_gpt_4o_mini)

    result = filter_patents_by_subclass(patent_ds, subclass_id, limit)
    print(str(len(result)) + " patents were found for the subclass "+subclass_id+".")

    for patent in result:
      id += 1
      patent_no = patent['No']
      title = patent['Title']
      subclass_labels = patent['Subclass_labels']
      abstract = patent['Abstract']

      #text = patent['Title']+" "+patent['Abstract']+" "+subclass_genai_desc
      text = patent['Title']+" "+patent['Abstract']
      embedding = model.encode(text).tolist()

      points.append(PointStruct(id=id, vector=embedding,
                                payload={'no': patent_no,
                                         'subclass_labels': subclass_id,
                                         'title': title,
                                         'abstract': abstract}))
                                         #'subclass': subclass_genai_desc}))

      if ((id % 50 ) == 0):
        operation_info = client.upsert(
                         collection_name=index_name,
                         wait=True,
                         points=points
                    )
        points.clear()
        print(" -> {} patent were indexed!!!".format(id))

if len(points) != 0:
  operation_info = client.upsert(
                         collection_name=index_name,
                         wait=True,
                         points=points
                    )
  print("{} patent subclasses were indexed!!!".format(id))
print("Indexing was finished!!!")

110 patents were found for the subclass A01B.
 -> 50 patent were indexed!!!
 -> 100 patent were indexed!!!
75 patents were found for the subclass A01C.
 -> 150 patent were indexed!!!
136 patents were found for the subclass A01D.
 -> 200 patent were indexed!!!
 -> 250 patent were indexed!!!
 -> 300 patent were indexed!!!
37 patents were found for the subclass A01F.
 -> 350 patent were indexed!!!
132 patents were found for the subclass A01G.
 -> 400 patent were indexed!!!
 -> 450 patent were indexed!!!
200 patents were found for the subclass A01H.
 -> 500 patent were indexed!!!
 -> 550 patent were indexed!!!
 -> 600 patent were indexed!!!
 -> 650 patent were indexed!!!
64 patents were found for the subclass A01J.
 -> 700 patent were indexed!!!
 -> 750 patent were indexed!!!
200 patents were found for the subclass A01K.
 -> 800 patent were indexed!!!
 -> 850 patent were indexed!!!
 -> 900 patent were indexed!!!
 -> 950 patent were indexed!!!
7 patents were found for the subclass A01L.
104